# Bangkok liveability indicators
Carl Higgs
10 December 2018

## OSM set up
Using OpenStreetMap as a source for 
* complete road network, and 
* a pedestrian 'walk/cycle'network. 
* calculating intersection density for the pedestrian network

In [2]:
# Libraries used for OSM conversion
import os
import sys
import subprocess as sp
from datetime import datetime

# Libraries used for OSMnx analyses and output
import networkx as nx
import osmnx as ox
import requests
import fiona
ox.config(use_cache=True, log_console=True)
ox.__version__

from shapely.geometry import shape, MultiPolygon, Polygon

# define pedestrian network custom filter (based on OSMnx 'walk' network type, without the cycling exclusion)
pedestrian = (
             '["area"!~"yes"]' 
             '["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]'
             '["foot"!~"no"]'  
             '["service"!~"private"]' 
             '["access"!~"private"]'
             )

## Extract OSM using 10km buffered study region .poly files

The idea with this step is to extract OSM for each buffered study region, and then this file would be used to build the all non-private roads, and pedestrian roads networks.  However there is apparently a bug with the OSMnx 'graph_from_file' command; as such, OSMnx networks were constructed using the OSM version current on Overpass.de at date of processing (2/10/2018).

In [5]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of source OSM file
osm_dir = 'D:/osm/planet_archives/planet-latest_20181001.osm.pbf'

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/21Cities/OSM_Roads'

# conversion settings
exe = 'osmconvert64-0.8.8p.exe'
exepath = 'D:/osm/'
osm_format = 'osm'

# output suffix
suffix = '_20181001'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".poly"):
           # Extract OSM
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = '{root}/{filename}{suffix}.{osm_format}'.format(root = root,
                                                                         filename = filename,
                                                                         suffix = suffix,
                                                                         osm_format = osm_format)
           command = '{osmconvert} {osm} -B={poly} -o={studyregion}'.format(osmconvert = exe, 
                                                                            osm = osm_dir,
                                                                            poly = fullfile,
                                                                            studyregion = studyregion)
           sp.call(command, shell=True, cwd=exepath)
           count+=1
           print(' Extraction of .osm file for {} complete in {:.1f} minutes.'.format(filename,
                                                                                  (datetime.now() - subtime).total_seconds()/60))
            
print('\nExtracted (or attempted to extract) {} OSM portions.'.format(count))            
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))

 Extraction of .osm file for AlburyWodonga complete in 19.1 minutes.
 Extraction of .osm file for Ballarat complete in 19.2 minutes.
 Extraction of .osm file for Bendigo complete in 19.3 minutes.
 Extraction of .osm file for Cairns complete in 19.3 minutes.
 Extraction of .osm file for adelaide_gccsa_2016_10000m_0 complete in 19.5 minutes.
 Extraction of .osm file for bris_gccsa_2016_10000m_0 complete in 19.9 minutes.
 Extraction of .osm file for canberra_gccsa_2016_10000m_0 complete in 20.0 minutes.
 Extraction of .osm file for darwin_gccsa_2016_10000m_0 complete in 20.1 minutes.
 Extraction of .osm file for hobart_gccsa_2016_10000m_0 complete in 19.8 minutes.
 Extraction of .osm file for melb_gccsa_2016_10000m_0 complete in 20.4 minutes.
 Extraction of .osm file for mitchell_lga_2016_10000m_0 complete in 20.9 minutes.
 Extraction of .osm file for perth_gccsa_2016_10000m_0 complete in 21.4 minutes.
 Extraction of .osm file for syd_gccsa_2016_10000m_0 complete in 21.5 minutes.
 Extract

## Get networks and save as graphs - Regional cities

In [22]:
count = 0
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10kmBuff.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('Buff','')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))        

Saved graph object and shapefile for AlburyWodonga_10km in 1.3 minutes.
Saved graph object and shapefile for Ballarat_10km in 1.4 minutes.
Saved graph object and shapefile for Bendigo_10km in 1.0 minutes.
Saved graph object and shapefile for Cairns_10km in 0.9 minutes.
Saved graph object and shapefile for Geelong_10km in 2.1 minutes.
Saved graph object and shapefile for GoldCoast_10km in 4.6 minutes.
Saved graph object and shapefile for Launceston_10km in 0.8 minutes.
Saved graph object and shapefile for Mackay_10km in 0.5 minutes.
Saved graph object and shapefile for Newcastle_10km in 3.1 minutes.
Saved graph object and shapefile for SunshineCoast_10km in 2.3 minutes.
Saved graph object and shapefile for Toowoomba_10km in 1.0 minutes.
Saved graph object and shapefile for Townsville_10km in 0.9 minutes.
Saved graph object and shapefile for WesternSydney_10km in 10.7 minutes.
Saved graph object and shapefile for Wollongong_10km in 2.5 minutes.
Elapsed time was 2574.6 minutes


In [21]:
count = 0
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10kmBuff.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('Buff','')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian)
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))   

Saved graph object and shapefile for AlburyWodonga_10km in 1.0 minutes.
Saved graph object and shapefile for Ballarat_10km in 1.1 minutes.
Saved graph object and shapefile for Bendigo_10km in 0.8 minutes.
Saved graph object and shapefile for Cairns_10km in 0.7 minutes.
Saved graph object and shapefile for Geelong_10km in 1.6 minutes.
Saved graph object and shapefile for GoldCoast_10km in 4.4 minutes.
Saved graph object and shapefile for Launceston_10km in 0.8 minutes.
Saved graph object and shapefile for Mackay_10km in 0.6 minutes.
Saved graph object and shapefile for Newcastle_10km in 3.2 minutes.
Saved graph object and shapefile for SunshineCoast_10km in 2.2 minutes.
Saved graph object and shapefile for Toowoomba_10km in 1.1 minutes.
Saved graph object and shapefile for Townsville_10km in 0.8 minutes.
Saved graph object and shapefile for WesternSydney_10km in 10.5 minutes.
Saved graph object and shapefile for Wollongong_10km in 2.7 minutes.
Elapsed time was 1628.1 minutes


## Get networks and save as graphs - GCCSAs

In [24]:
count = 0
# this is to extract networks for GCCSAs; new shape file naming convention is used
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10000m_epsg4326.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('10000m_epsg4326','10km')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian)
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))    

Saved graph object and shapefile for adelaide_gccsa_2016_10km in 15.5 minutes.
Saved graph object and shapefile for bris_gccsa_2016_10km in 27.2 minutes.
Saved graph object and shapefile for canberra_gccsa_2016_10km in 11.9 minutes.
Saved graph object and shapefile for darwin_gccsa_2016_10km in 1.8 minutes.
Saved graph object and shapefile for hobart_gccsa_2016_10km in 2.8 minutes.
Saved graph object and shapefile for melb_gccsa_2016_10km in 44.8 minutes.
Saved graph object and shapefile for mitchell_lga_2016_10km in 2.5 minutes.
Saved graph object and shapefile for perth_gccsa_2016_10km in 20.2 minutes.
Saved graph object and shapefile for syd_gccsa_2016_10km in 32.1 minutes.
Elapsed time was 2820.0 minutes


## Repeat OSM extraction for Bris using not most current data to compare with Julianna's work

In [25]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of source OSM file - dated 8 Feb 2018
osm_dir = 'D:/osm/australia-oceania-latest.osm.pbf'

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/21Cities/OSM_Roads/GCCSAs/Brisbane'

# conversion settings
exe = 'osmconvert64-0.8.8p.exe'
exepath = 'D:/osm/'
osm_format = 'osm'

# output suffix
suffix = '_20180208'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".poly"):
           # Extract OSM
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = '{root}/{filename}{suffix}.{osm_format}'.format(root = root,
                                                                         filename = filename,
                                                                         suffix = suffix,
                                                                         osm_format = osm_format)
           command = '{osmconvert} {osm} -B={poly} -o={studyregion}'.format(osmconvert = exe, 
                                                                            osm = osm_dir,
                                                                            poly = fullfile,
                                                                            studyregion = studyregion)
           sp.call(command, shell=True, cwd=exepath)
           count+=1
           print(' Extraction of .osm file for {} complete in {:.1f} minutes.'.format(filename,
                                                                                  (datetime.now() - subtime).total_seconds()/60))
            
print('\nExtracted (or attempted to extract) {} OSM portions.'.format(count))            
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))

 Extraction of .osm file for bris_gccsa_2016_10000m complete in 0.3 minutes.

Extracted (or attempted to extract) 1 OSM portions.
Elapsed time was 0.3 minutes
